In [1]:
from joblib import dump

In [2]:
import os
import pickle
import tensorflow as tf
import numpy as np
from tqdm.notebook import tqdm
from tensorflow.keras.applications.vgg16 import VGG16,preprocess_input
from tensorflow.keras.preprocessing.image import load_img,img_to_array
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model
from tensorflow.keras.utils import to_categorical,plot_model
from tensorflow.keras.layers import Input,Dense,LSTM,Embedding,Dropout,add

2025-01-05 09:45:42.977314: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1736070343.094832    2442 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1736070343.130658    2442 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-01-05 09:45:43.416185: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
###EXtract Image Featurs
#load VGG16 model
model=VGG16()
#restructure Model
model=Model(inputs=model.inputs,outputs=model.layers[-2].output)
#summarize
print(model.summary())

I0000 00:00:1736070349.301457    2442 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 2248 MB memory:  -> device: 0, name: NVIDIA GeForce GTX 1650, pci bus id: 0000:01:00.0, compute capability: 7.5


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv1 (Conv2D)           │ (None, 224, 224, 64)   │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv2 (Conv2D)           │ (None, 224, 224, 64)   │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_pool (MaxPooling2D)      │ (None, 112, 112, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv1 (Conv2D)           │ (None, 112, 112, 128)  │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv2 (Conv2D)           │ (None, 112, 112, 128)  │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_pool (MaxPooling2D)      │ (None, 56, 56, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv1 (Conv2D)           │ (None, 56, 56, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv2 (Conv2D)           │ (None, 56, 56, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv3 (Conv2D)           │ (None, 56, 56, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_pool (MaxPooling2D)      │ (None, 28, 28, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv1 (Conv2D)           │ (None, 28, 28, 512)    │     1,180,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv2 (Conv2D)           │ (None, 28, 28, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv3 (Conv2D)           │ (None, 28, 28, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_pool (MaxPooling2D)      │ (None, 14, 14, 512)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv1 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv2 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv3 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_pool (MaxPooling2D)      │ (None, 7, 7, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 25088)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ fc1 (Dense)                     │ (None, 4096)           │   102,764,544 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ fc2 (Dense)                     │ (None, 4096)           │    16,781,312 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 134,260,544 (512.16 MB)

 Trainable params: 134,260,544 (512.16 MB)

 Non-trainable params: 0 (0.00 B)

None


In [4]:
#Extract Features from Image
features={}
directory='./Images'

'''for img_name in tqdm(os.listdir(directory)):
    #load image form file
    
    img_path=directory+'/'+img_name
    image=load_img(img_path,target_size=(224,224))
    #image.show()
    
    
    #convert image to numpy array
    image=img_to_array(image)
    #reshape data for model
    image=image.reshape((1,image.shape[0],image.shape[1], image.shape[2]))
    #preprocess image for VGG16
    image=preprocess_input(image)
    #extract features
    feature=model.predict(image,verbose=0)
    #get image ID
    image_id=img_name.split('.')[0]
    
    
    features[image_id]=feature
    
    '''

"for img_name in tqdm(os.listdir(directory)):\n    #load image form file\n    \n    img_path=directory+'/'+img_name\n    image=load_img(img_path,target_size=(224,224))\n    #image.show()\n    \n    \n    #convert image to numpy array\n    image=img_to_array(image)\n    #reshape data for model\n    image=image.reshape((1,image.shape[0],image.shape[1], image.shape[2]))\n    #preprocess image for VGG16\n    image=preprocess_input(image)\n    #extract features\n    feature=model.predict(image,verbose=0)\n    #get image ID\n    image_id=img_name.split('.')[0]\n    \n    \n    features[image_id]=feature\n    \n    "

In [5]:

#store features in pickle
'''data = features
# Save the data to a pickle file
with open('features.pkl', 'wb') as f:  # 'wb' ensures binary write mode
    pickle.dump(data, f, protocol=pickle.HIGHEST_PROTOCOL)  # Use highest protocol for compatibility'''

"data = features\n# Save the data to a pickle file\nwith open('features.pkl', 'wb') as f:  # 'wb' ensures binary write mode\n    pickle.dump(data, f, protocol=pickle.HIGHEST_PROTOCOL)  # Use highest protocol for compatibility"

In [6]:
#load features from pickle
try:
    with open('./vars/features.pkl', 'rb') as f:
        features = pickle.load(f)
        print("Features loaded successfully.")
except EOFError:
    print("The file is empty or corrupted. Please regenerate the pickle file.")

Features loaded successfully.


In [7]:
#load captions data
with open('./vars/captions.txt','r') as f:
    next(f)
    captions_doc=f.read()
    

In [8]:
#captions_doc
#features

In [9]:
#create mapping of image to captions
mapping={}
#process lines
for line in tqdm(captions_doc.split('\n')):
    #split the line by comma
    tokens=line.split(',')
    if len(line)<2:
        continue
    image_id,caption=tokens[0],tokens[1:]
    #remove extension from image id
    image_id=image_id.split('.')[0]
    #convert caption list to string
    caption=" ".join(caption)
    #create list if needed
    if image_id not in mapping:
        mapping[image_id]=[]
    mapping[image_id].append(caption)

  0%|          | 0/40456 [00:00<?, ?it/s]

In [10]:
len(mapping)

8091

In [11]:
#Preprocess Text Data
def clean(mapping):
    for key,captions in mapping.items():
        for i in range(len(captions)):
            #take one caption at a time
            caption=captions[i]
            #preprocessing steps
            caption=caption.lower()#convert to lower case
            caption=caption.replace('[^A-Za-z]','')#removing special characters and digits
            caption=caption.replace(r'\s+',' ')#removing multiple spaces
            
            #add start and end tags
            caption='startseq '+" ".join([word for word in caption.split() if len(word)>1])+' endseq'
            captions[i]=caption
            
        

In [12]:
#before preprocess 
mapping['1000268201_693b08cb0e']

['A child in a pink dress is climbing up a set of stairs in an entry way .',
 'A girl going into a wooden building .',
 'A little girl climbing into a wooden playhouse .',
 'A little girl climbing the stairs to her playhouse .',
 'A little girl in a pink dress going into a wooden cabin .']

In [13]:
#preprocess the text
clean(mapping)

In [14]:
#after Preprocess
mapping['1000268201_693b08cb0e']

['startseq child in pink dress is climbing up set of stairs in an entry way endseq',
 'startseq girl going into wooden building endseq',
 'startseq little girl climbing into wooden playhouse endseq',
 'startseq little girl climbing the stairs to her playhouse endseq',
 'startseq little girl in pink dress going into wooden cabin endseq']

In [15]:
dump(mapping,"./vars/Mapping.joblib")

['./vars/Mapping.joblib']

In [16]:
all_captions=[]
for key in mapping:
    for caption in mapping[key]:
        all_captions.append(caption)

In [17]:
len(all_captions)

40455

In [18]:
all_captions[:10]

['startseq child in pink dress is climbing up set of stairs in an entry way endseq',
 'startseq girl going into wooden building endseq',
 'startseq little girl climbing into wooden playhouse endseq',
 'startseq little girl climbing the stairs to her playhouse endseq',
 'startseq little girl in pink dress going into wooden cabin endseq',
 'startseq black dog and spotted dog are fighting endseq',
 'startseq black dog and tri-colored dog playing with each other on the road endseq',
 'startseq black dog and white dog with brown spots are staring at each other in the street endseq',
 'startseq two dogs of different breeds looking at each other on the road endseq',
 'startseq two dogs on pavement moving toward each other endseq']

In [19]:
#tokenize the text
tokenizer=Tokenizer()
tokenizer.fit_on_texts(all_captions)
dump(tokenizer,'./vars/Tokenizer.joblib')
vocab_size=len(tokenizer.word_index)+1
vocab_size

8485

In [20]:
#get max length of the caption available
max_len=max(len(caption.split()) for caption in all_captions)
max_len

35

In [21]:
#Train Test Split
image_ids=list(mapping.keys())
split=int(len(image_ids)*0.90)
train=image_ids[:split]
test=image_ids[split:]

In [22]:
# create data generator to get data in batch (avoids session crash)
def data_generator(data_keys, mapping, features, tokenizer, max_length, vocab_size, batch_size):
    # loop over images
    X1, X2, y = list(), list(), list()
    n = 0
    while 1:
        for key in data_keys:
            n += 1
            captions = mapping[key]
            # process each caption
            for caption in captions:
                # encode the sequence
                seq = tokenizer.texts_to_sequences([caption])[0]
                # split the sequence into X, y pairs
                for i in range(1, len(seq)):
                    # split into input and output pairs
                    in_seq, out_seq = seq[:i], seq[i]
                    # pad input sequence
                    in_seq = pad_sequences([in_seq], maxlen=max_length, padding='post')[0]
                    # encode output sequence
                    out_seq = to_categorical([out_seq], 	num_classes=vocab_size)[0]
                    # store the sequences
                    X1.append(features[key][0])
                    X2.append(in_seq)
                    y.append(out_seq)
            if n == batch_size:
                X1, X2, y = np.array(X1), np.array(X2), np.array(y)
                yield {'image': X1, 'text': X2}, y
                X1, X2, y = list(), list(), list()
                n = 0

In [23]:
# encoder model
# image feature layers
'''inputs1 = Input(shape=(4096,), name="image")
fe1 = Dropout(0.4)(inputs1)
fe2 = Dense(256, activation='relu')(fe1)
# sequence feature layers
inputs2 = Input(shape=(max_len,), name="text")
se1 = Embedding(vocab_size, 256, mask_zero=True)(inputs2)
se2 = Dropout(0.4)(se1)
se3 = LSTM(256)(se2)

# decoder model
decoder1 = add([fe2, se3])
decoder2 = Dense(256, activation='relu')(decoder1)
outputs = Dense(vocab_size, activation='softmax')(decoder2)

model = Model(inputs=[inputs1, inputs2], outputs=outputs)
model.compile(loss='categorical_crossentropy', optimizer='adam')

# plot the model
plot_model(model, show_shapes=True)'''

'inputs1 = Input(shape=(4096,), name="image")\nfe1 = Dropout(0.4)(inputs1)\nfe2 = Dense(256, activation=\'relu\')(fe1)\n# sequence feature layers\ninputs2 = Input(shape=(max_len,), name="text")\nse1 = Embedding(vocab_size, 256, mask_zero=True)(inputs2)\nse2 = Dropout(0.4)(se1)\nse3 = LSTM(256)(se2)\n\n# decoder model\ndecoder1 = add([fe2, se3])\ndecoder2 = Dense(256, activation=\'relu\')(decoder1)\noutputs = Dense(vocab_size, activation=\'softmax\')(decoder2)\n\nmodel = Model(inputs=[inputs1, inputs2], outputs=outputs)\nmodel.compile(loss=\'categorical_crossentropy\', optimizer=\'adam\')\n\n# plot the model\nplot_model(model, show_shapes=True)'

In [24]:
from tensorflow.keras.models import load_model
epochs = 20
batch_size = 32
steps = len(train) // batch_size
model=load_model("./model/best_model_120_epochs.keras")
for i in range(epochs):
    # create data generator
    generator = data_generator(train, mapping, features, tokenizer, max_len, vocab_size, batch_size)
    print("epoch : %d" %i)
    # fit for one epoch
    model.fit(generator, epochs=1, steps_per_epoch=steps, verbose=1)

epoch : 0


I0000 00:00:1736070357.975088    2863 cuda_dnn.cc:529] Loaded cuDNN version 90300


227/227 ━━━━━━━━━━━━━━━━━━━━ 26s 107ms/step - loss: 1.4095
epoch : 1
227/227 ━━━━━━━━━━━━━━━━━━━━ 24s 103ms/step - loss: 1.4088
epoch : 2
227/227 ━━━━━━━━━━━━━━━━━━━━ 26s 113ms/step - loss: 1.4075
epoch : 3
227/227 ━━━━━━━━━━━━━━━━━━━━ 24s 105ms/step - loss: 1.3999
epoch : 4
227/227 ━━━━━━━━━━━━━━━━━━━━ 24s 106ms/step - loss: 1.3976
epoch : 5
227/227 ━━━━━━━━━━━━━━━━━━━━ 26s 115ms/step - loss: 1.3923
epoch : 6
227/227 ━━━━━━━━━━━━━━━━━━━━ 25s 108ms/step - loss: 1.3871
epoch : 7
227/227 ━━━━━━━━━━━━━━━━━━━━ 25s 110ms/step - loss: 1.3846
epoch : 8
227/227 ━━━━━━━━━━━━━━━━━━━━ 27s 119ms/step - loss: 1.3860
epoch : 9
227/227 ━━━━━━━━━━━━━━━━━━━━ 27s 120ms/step - loss: 1.3806
epoch : 10
227/227 ━━━━━━━━━━━━━━━━━━━━ 26s 115ms/step - loss: 1.3793
epoch : 11
227/227 ━━━━━━━━━━━━━━━━━━━━ 25s 108ms/step - loss: 1.3764
epoch : 12
227/227 ━━━━━━━━━━━━━━━━━━━━ 25s 111ms/step - loss: 1.3743
epoch : 13
227/227 ━━━━━━━━━━━━━━━━━━━━ 24s 106ms/step - loss: 1.3687
epoch : 14
227/227 ━━━━━━━━━━━━━━━━━━━━ 

In [25]:

#os.makedirs("model", exist_ok=True)
model.save("./model/best_model_150_epochs.keras")


In [26]:
#generate Captions for Image
def idx_to_word(integer,tokenizer):
    for word,index in tokenizer.word_index.items():
        if index == integer: 
            return word
    return none
    

In [27]:
#generate caption for an image
def predict_caption(model,image,tokenizer,max_length):
    #add start tag for generation
    in_text='startseq'
    #iterate over maxlength of sequence
    for i in range(max_length):
        #incode input sequence
        sequence=tokenizer.texts_to_sequences([in_text])[0]
        #pad the sequence
        sequence=pad_sequences([sequence],max_length)
        #predict next word
        yhat=model.predict([image,sequence],verbose=0)
        #convert index with high probability
        yhat=np.argmax(yhat)
        #convert index to word
        word=idx_to_word(yhat,tokenizer)
        #stop if word not found
        if word is None:
            break
        #append word as input for generation next word
        in_text +=" "+word
        #stop if we reach end tag
        if word =='endseq':
            break
    return in_text
    

In [28]:

from nltk.translate.bleu_score import corpus_bleu
#validate with test data
actual,predicted=[],[]
for key in tqdm(test):
    #get actudal caption
    captions=mapping[key]
    #predict the caption for image
    y_pred=predict_caption(model,features[key],tokenizer,max_len)
    #split into words
    actual_captions=[caption.split() for caption in captions]
    y_pred=y_pred.split()
    actual.append(actual_captions)
    predicted.append(y_pred)
#Calculate BLEU Score
print("Corpus will be started")
print("BLEU-1: %f" %corpus_bleu(actual,predicted,weights=(1.0,0,0,0)))
print("BLEU-1: %f" %corpus_bleu(actual,predicted,weights=(0.5,0.5,0,0)))

'from nltk.translate.bleu_score import corpus_bleu\n#validate with test data\nactual,predicted=[],[]\nfor key in tqdm(test):\n    #get actudal caption\n    captions=mapping[key]\n    #predict the caption for image\n    y_pred=predict_caption(model,features[key],tokenizer,max_len)\n    #split into words\n    actual_captions=[caption.split() for caption in captions]\n    y_pred=y_pred.split()\n    actual.append(actual_captions)\n    predicted.append(y_pred)\n#Calculate BLEU Score\nprint("Corpus will be started")\nprint("BLEU-1: %f" %corpus_bleu(actual,predicted,weights=(1.0,0,0,0)))\nprint("BLEU-1: %f" %corpus_bleu(actual,predicted,weights=(0.5,0.5,0,0)))'

In [29]:
 
model3=tf.keras.models.load_model("./model/best_model_90_epochs.keras")

In [30]:
##Visualize results\
from PIL import Image
import matplotlib.pyplot as plt
def generate_caption(image_name,model):
    #load Image
    #image_name="1002674143_1b742ab4b8.jpg"
    image_id=image_name.split('.')[0]
    image_path=os.path.join("Images",image_name)
    image=Image.open(image_path)
    
    captions=mapping[image_id]
    print("---------------------------Original Caption--------------------------")
    for caption in captions:
        print(caption)
    
    #predict the caption
    y_pred=predict_caption(model,features[image_id],tokenizer,max_len)
    
    print("--------------------------Predicted Caption----------------")
    print(y_pred)
    plt.imshow(image)

In [31]:
generate_caption("3747543364_bf5b548527.jpg",model3)

---------------------------Original Caption--------------------------
startseq three girls do back flips and cartwheels in the surf endseq
startseq three girls do handstands at the beach endseq
startseq three girls doing back flips on the beach endseq
startseq three little girls cartwheel in the ocean endseq
startseq three young girls playing on the beach endseq


2025-01-05 09:54:56.275000: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous is aborting with status: INVALID_ARGUMENT: assertion failed: [You are passing a RNN mask that does not correspond to right-padded sequences, while using cuDNN, which is not supported. With cuDNN, RNN masks can only be used for right-padding, e.g. `[[True, True, False, False]]` would be a valid mask, but any mask that isn\'t just contiguous `True`\'s on the left and contiguous `False`\'s on the right would be invalid. You can pass `use_cudnn=False` to your RNN layer to stop using cuDNN (this may be slower).]
	 [[{{node functional_1_1/lstm_1/Assert/Assert}}]]


InvalidArgumentError: Graph execution error:

Detected at node functional_1_1/lstm_1/Assert/Assert defined at (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main

  File "<frozen runpy>", line 88, in _run_code

  File "/home/tanuj/anacondaWSL/lib/python3.12/site-packages/ipykernel_launcher.py", line 17, in <module>

  File "/home/tanuj/anacondaWSL/lib/python3.12/site-packages/traitlets/config/application.py", line 1075, in launch_instance

  File "/home/tanuj/anacondaWSL/lib/python3.12/site-packages/ipykernel/kernelapp.py", line 701, in start

  File "/home/tanuj/anacondaWSL/lib/python3.12/site-packages/tornado/platform/asyncio.py", line 205, in start

  File "/home/tanuj/anacondaWSL/lib/python3.12/asyncio/base_events.py", line 641, in run_forever

  File "/home/tanuj/anacondaWSL/lib/python3.12/asyncio/base_events.py", line 1986, in _run_once

  File "/home/tanuj/anacondaWSL/lib/python3.12/asyncio/events.py", line 88, in _run

  File "/home/tanuj/anaconda